## CS431/631 Big Data Infrastructure
### Fall 2019 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** 
* **ID:** 

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [1]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
import itertools as ite
tokens = []
tokens_b = []
# Now, let's tokenize Shakespeare's plays
with open('Shakespeare.txt') as f:
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)
        tokens.append(t)
        
# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs
# using to set to get distinct tokens
res_a = set(x for l in tokens for x in l)
print('a) Number of distinct tokens:',len(res_a))

for token in tokens:
    tokens_b.append(list(ite.permutations(set(token),2)))

res_b = set(x for l in tokens_b for x in l)
print('b) Number of distinct token pairs:',len(res_b))

a) Number of distinct tokens: 25975
b) Number of distinct token pairs: 1969760


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


In [2]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log
import itertools as ite
from collections import Counter

###################################################################################################################
#  replace this with your PMI analysis code, so that you can support the user interface below
#  it should read and tokenize Shakespeare.txt, and store enough information in Python data structures
#  to allow you to answer the PMI queries below
###################################################################################################################
#Creating list and dictionaries to store all the tokens and token pairs
tokens_pair = []
tokens_single = []
n = 0
single_dict = {}
pair_dict = {}
with open('Shakespeare.txt') as f:
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)
        #store unique words in the list
        tokens_single.append(list(set(t)))
        #count no. of lines
        n += 1
        #storing unique token pair for each line
        tokens_pair.append(list(ite.permutations(set(t),2)))

#storing unique words and its frequency in a dictionary
single_dict = dict(Counter(x for sublist in tokens_single for x in sublist))
#storing unique tokens and their frequency in a dictionary
pair_dict = dict(Counter(x for sublist in tokens_pair for x in sublist))

#function for two token query
def PMI_token_pair(x,y,single_dict,pair_dict,n):
    n_pair = pair_dict[(x,y)]
    n_x = single_dict[x]
    n_y = single_dict[y]
    PMI = log((n_pair*n)/(n_x*n_y),10)
    return n_pair,n_x,n_y,PMI

#function for one token query
def PMI_one_token(x,single_dict,pair_dict,n,threshold):
    t_dict = {}
    n_x = 0
    n_y = 0
    n_pair = 0
    x_list = []
    #storing the pairs in a list for all tokens starting with q_token[0]
    for item in tokens_pair:
        for i in item:
            if x == i[0]:
                x_list.append(i)
    #iterating through the list to fetch each and every pair
    for pair in set(x_list):
        y = pair[1]
        n_pair = pair_dict[(x,y)]
        #checking the threshold condition for co-occurence of the words
        if n_pair >= threshold:
            n_x = single_dict[x]
            n_y = single_dict[y]
            PMI = log((n_pair*n)/(n_x*n_y),10)
        else:
            PMI = -9999      #storing a default value, if the threshold condition is not met
        #storing the PMI value for every token
        t_dict[pair] = [PMI,n_pair]
    #sorting the dictionary based on the PMI values
    sorted_dict = sorted(t_dict.items(), key=lambda kv: kv[1],reverse=True)
    return sorted_dict,n_x
###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################

while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
        one_pair_PMI,n_x = PMI_one_token(q_tokens[0],single_dict,pair_dict,n,threshold)
        # Put code here to answer a One-Token Query with token q_tokens[0] and the specified threshold,
        # and output the result.
        # The print() statements below exist to show you the desired output format.
        # Replace them with your own output code, which should produce results in a similar format.
        print("  n({0}) = {1:03d}".format(q_tokens[0],n_x))
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))
        print("    n({0},{3}) = {1},  PMI({0},{3}) = {2:.3f}".format(q_tokens[0],one_pair_PMI[0][1][1],one_pair_PMI[0][1][0],one_pair_PMI[0][0][1]))    
        print("    n({0},{3}) = {1},  PMI({0},{3}) = {2:.3f}".format(q_tokens[0],one_pair_PMI[1][1][1],one_pair_PMI[1][1][0],one_pair_PMI[1][0][1]))
        print("    n({0},{3}) = {1},  PMI({0},{3}) = {2:.3f}".format(q_tokens[0],one_pair_PMI[2][1][1],one_pair_PMI[2][1][0],one_pair_PMI[2][0][1]))
        print("    n({0},{3}) = {1},  PMI({0},{3}) = {2:.3f}".format(q_tokens[0],one_pair_PMI[3][1][1],one_pair_PMI[3][1][0],one_pair_PMI[3][0][1]))
        print("    n({0},{3}) = {1},  PMI({0},{3}) = {2:.3f}".format(q_tokens[0],one_pair_PMI[4][1][1],one_pair_PMI[4][1][0],one_pair_PMI[4][0][1]))
        # in the above, all XXX values should be at least as large as the threshold

    elif len(q_tokens) == 2:
        # Put code here to answer a Two-Token Query with tokens q_tokens[0] and q_tokens[1]
        # As was the case for the One-Token query, the print statements below show the desired output format
        # Replace them with your own output code
        n_pair,n_x,n_y,PMI = PMI_token_pair(q_tokens[0],q_tokens[1],single_dict,pair_dict,n)
        print("  n({0},{1}) = {2:03d}".format(q_tokens[0],q_tokens[1],n_pair))
        print("  PMI({0},{1}) = {2:.3f}".format(q_tokens[0],q_tokens[1],PMI))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


Input 1 or 2 space-separated tokens (return to quit): the
Input a positive integer frequency threshold: 100
  n(the) = 24300
  high PMI tokens with respect to the (threshold: 100):
    n(the,king's) = 146,  PMI(the,king's) = 0.649
    n(the,moon) = 101,  PMI(the,moon) = 0.593
    n(the,world) = 482,  PMI(the,world) = 0.592
    n(the,sun) = 169,  PMI(the,sun) = 0.578
    n(the,field) = 140,  PMI(the,field) = 0.567
Input 1 or 2 space-separated tokens (return to quit): peace fire
  n(peace,fire) = 001
  PMI(peace,fire) = -0.028
Input 1 or 2 space-separated tokens (return to quit): william shakespeare
  n(william,shakespeare) = 038
  PMI(william,shakespeare) = 3.005
Input 1 or 2 space-separated tokens (return to quit): 


---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:

*As the input grows the Execution time will increase significantly.The program will run a lot slower. When the file size is increased the data also increases, eventually the local disk will run out of storage and the program will crash. Even if we try to store the data in cache, it will only store upto certain extent and later on it will try to use the local disk space.
Also the task will process sequentially, which will lead to slower computation. In the above case itself for the file size of 5Mb it created approx. 2 Millions token pairs, if the file size is increased by 10 or 100 times the token pairs will increase exponetially. Resulting in shortage of disk space.*


---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.